In [1]:
#pip install pycryptodome
#pip install sympy

In [6]:
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Util import number
from sympy.ntheory.factor_ import totient
import binascii
import struct
import random

prime_base = 8
# This can be either generated by existing library (1) or calculated manually from the formulas (2)

"""
# Option 1
keyPair = RSA.generate(1024)
 
pubKey = keyPair.publickey()
print(f"Public key:  (n={hex(pubKey.n)}, e={hex(pubKey.e)})")
pubKeyPEM = pubKey.exportKey()
print(pubKeyPEM.decode('ascii'))
 
print(f"Private key: (n={hex(pubKey.n)}, d={hex(keyPair.d)})")
privKeyPEM = keyPair.exportKey()
print(privKeyPEM.decode('ascii'))
"""

#Option 2
p = number.getPrime(prime_base)
q = number.getPrime(prime_base)
n = p*q
n_tot = totient(n)
e = random.randint(3, 31)
while n_tot % e == 0:
    e = random.randint(3, 31)
    
# Modular multiplication inverse formula to calculate d, taken from internet.
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)
def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m
d = modinv(e, n_tot)

print("Prime base:",prime_base)
print("P:",p)
print("Q:",q)
print("N:",n)
print("E:",e)
print("D:",d)
print("Totient(N):",n_tot)
key1 = [n,e, prime_base]
key2 = [n,d, prime_base]

Prime base: 8
P: 131
Q: 167
N: 21877
E: 19
D: 5679
Totient(N): 21580


In [7]:
# Encryption for Strings

def encrypt(message, key):
    n = key[0]
    e = key[1]
    prime_base = key[2]
    enc_base = int(prime_base/2)
    dec_base = int(prime_base/4)

    # Step 1 - Converting the input into binary/ascii/hex
    msg_string = message
    msg_ascii = [ord(c) for c in msg_string]
    msg_hex = [intToHexFormatter(c,dec_base) for c in msg_ascii]

    # Step 2 - Hex to hex string
    msg_hex_string = "0x"
    for c in msg_hex:
        msg_hex_string += c[2:]

    # Step 3 - Taking the power of input so that enc_msg = (msg ^ e) % n
    enc_msg_int = []
    for c in msg_ascii:
        temp = pow(c,e) % n
        enc_msg_int.append(temp)

    # Step 4 - Converting the encrypted input into hex
    enc_msg_hex = []
    for c in enc_msg_int:
        temp = intToHexFormatter(c, enc_base)
        enc_msg_hex.append(temp)

    # Step 5 - Encrypted hex to encrypted hex string
    enc_msg_hex_string = "0x"
    for c in enc_msg_hex:
        enc_msg_hex_string += c[2:]

    # Step 6 - Output
    print("Original message information")
    print("\nMessage:",msg_string)
    print("\nMessage in ASCII format:",msg_ascii)
    print("\nMessage in Hex (ascii to hex) format:",msg_hex)
    print("\nMessage in complete hex string:",msg_hex_string)
    print("\n")
    print("\nEncrypted message information")
    print("\nEncrypted message in integer format:",enc_msg_int)
    print("\nEncrypted message in hex format:",enc_msg_hex)
    print("\nEncrypted messsage in complete hex string:",enc_msg_hex_string)
    
    return enc_msg_hex_string

def intToHexFormatter(entry, base):
    counter = 0
    temp = entry
    while temp >= 16:
        counter += 1
        temp = temp/16
    formattedHex = "0x"
    for i in range(int(base)-counter-1):
        formattedHex += "0"
    temp = hex(entry)
    formattedHex += temp[2:]
    return formattedHex

In [8]:
# Decryption for Strings

def decrypt(message, key):
    n = key[0]
    d = key[1]
    prime_base = key[2]
    enc_base = int(prime_base/2)
    dec_base = int(prime_base/4)
    # Step 1 - Converting the hex string into decryptable segments
    enc_msg_hex_string = message
    enc_msg_hex_length = len(enc_msg_hex_string[2:])
    enc_msg_received = []
    x = range(2,enc_msg_hex_length+1,enc_base)
    for c in x:
        temp = enc_msg_hex_string[c:c+enc_base]  #First two characters are 0x
        enc_msg_received.append(temp) 
    print(enc_msg_received)
    # Step 2 - Converting each encrypted hex segment into integers
    enc_msg_received_int = []
    for c in enc_msg_received:
        temp = int(c,16)
        enc_msg_received_int.append(temp)

    # Step 3 - Decrypting each integer segment
    dec_msg_int = []
    for c in enc_msg_received_int:
        temp = pow(c,d) % n
        dec_msg_int.append(temp)

    # Step 3 - Converting the decrypted segments into hex
    dec_msg_hex = []
    for c in dec_msg_int:
        dec_msg_hex.append(hex(c))

    # Step 4 - Decrypted hex to decrypted hex string
    dec_msg_hex_string = "0x"
    for c in dec_msg_hex:
        dec_msg_hex_string += c[2:]

    # Step 5 - Output
    print("\n\nBefore Decryption")
    print("\nEncrypted hex string received:",enc_msg_hex_string)
    print("\nEncrypted hex string converted divided into segments:",enc_msg_received)

    print("\n\nAfter Decryption")
    print("\nDecrypted message in integer format",dec_msg_int)
    print("\nDecrypted message in hex format",dec_msg_hex)
    print("\nDecrypted message in complete hex string:",dec_msg_hex_string)
    
    return dec_msg_hex_string

In [9]:
# Example
message = "Something to be made here"
x = encrypt(message, key1)
y = decrypt(x, key2)

Original message information

Message: Something to be made here

Message in ASCII format: [83, 111, 109, 101, 116, 104, 105, 110, 103, 32, 116, 111, 32, 98, 101, 32, 109, 97, 100, 101, 32, 104, 101, 114, 101]

Message in Hex (ascii to hex) format: ['0x53', '0x6f', '0x6d', '0x65', '0x74', '0x68', '0x69', '0x6e', '0x67', '0x20', '0x74', '0x6f', '0x20', '0x62', '0x65', '0x20', '0x6d', '0x61', '0x64', '0x65', '0x20', '0x68', '0x65', '0x72', '0x65']

Message in complete hex string: 0x536f6d657468696e6720746f206265206d6164652068657265



Encrypted message information

Encrypted message in integer format: [17152, 13358, 5354, 8486, 14948, 17411, 19295, 6285, 581, 17623, 14948, 13358, 17623, 5215, 8486, 17623, 5354, 13220, 6295, 8486, 17623, 17411, 8486, 7915, 8486]

Encrypted message in hex format: ['0x4300', '0x342e', '0x14ea', '0x2126', '0x3a64', '0x4403', '0x4b5f', '0x188d', '0x0245', '0x44d7', '0x3a64', '0x342e', '0x44d7', '0x145f', '0x2126', '0x44d7', '0x14ea', '0x33a4', '0x1897', '0x21

In [10]:
# Encryption reading file

with open('input/msg2.txt', 'rb') as f:
    message = f.read()
f.close()

enc_message = encrypt(message.decode(),key1)
    
with open('intermediate/msg2_encrypted.txt', 'wb') as f:
    f.write(str.encode(enc_message))
f.close()

print("\n\nEncrypted file has been successfully edited/created.")

Original message information

Message: Second message: Hakuna Matata

Message in ASCII format: [83, 101, 99, 111, 110, 100, 32, 109, 101, 115, 115, 97, 103, 101, 58, 32, 72, 97, 107, 117, 110, 97, 32, 77, 97, 116, 97, 116, 97]

Message in Hex (ascii to hex) format: ['0x53', '0x65', '0x63', '0x6f', '0x6e', '0x64', '0x20', '0x6d', '0x65', '0x73', '0x73', '0x61', '0x67', '0x65', '0x3a', '0x20', '0x48', '0x61', '0x6b', '0x75', '0x6e', '0x61', '0x20', '0x4d', '0x61', '0x74', '0x61', '0x74', '0x61']

Message in complete hex string: 0x5365636f6e64206d6573736167653a2048616b756e61204d6174617461



Encrypted message information

Encrypted message in integer format: [17152, 8486, 11197, 13358, 6285, 6295, 17623, 5354, 8486, 4406, 4406, 13220, 581, 8486, 7004, 17623, 7565, 13220, 20088, 4169, 6285, 13220, 17623, 8858, 13220, 14948, 13220, 14948, 13220]

Encrypted message in hex format: ['0x4300', '0x2126', '0x2bbd', '0x342e', '0x188d', '0x1897', '0x44d7', '0x14ea', '0x2126', '0x1136', '0x1136', '0

In [11]:
# Decryption reading file

with open('intermediate/msg2_encrypted.txt', 'rb') as f:
    message = f.read()
f.close()

dec_message = decrypt(message.decode(),key2)

# File conversions
dec_msg_hex_string = dec_message

dec_msg_hex_length = len(dec_msg_hex_string[2:len(dec_msg_hex_string)])
dec_msg_received = []

x = range(2,dec_msg_hex_length+1,2)
for c in x:
    temp = dec_msg_hex_string[c:c+2]  #First two characters are 0x
    dec_msg_received.append(temp)

# 1 - Hex to proper text
original_message = ""
for c in dec_msg_received:
    temp = chr(int(c,16))
    original_message += temp

with open('output/msg2_decrypted.txt', 'wb') as f:
    f.write(str.encode(original_message))
f.close()

print("\n\nDecrypted file has been successfully edited/created.")

['4300', '2126', '2bbd', '342e', '188d', '1897', '44d7', '14ea', '2126', '1136', '1136', '33a4', '0245', '2126', '1b5c', '44d7', '1d8d', '33a4', '4e78', '1049', '188d', '33a4', '44d7', '229a', '33a4', '3a64', '33a4', '3a64', '33a4']


Before Decryption

Encrypted hex string received: 0x430021262bbd342e188d189744d714ea21261136113633a4024521261b5c44d71d8d33a44e781049188d33a444d7229a33a43a6433a43a6433a4

Encrypted hex string converted divided into segments: ['4300', '2126', '2bbd', '342e', '188d', '1897', '44d7', '14ea', '2126', '1136', '1136', '33a4', '0245', '2126', '1b5c', '44d7', '1d8d', '33a4', '4e78', '1049', '188d', '33a4', '44d7', '229a', '33a4', '3a64', '33a4', '3a64', '33a4']


After Decryption

Decrypted message in integer format [83, 101, 99, 111, 110, 100, 32, 109, 101, 115, 115, 97, 103, 101, 58, 32, 72, 97, 107, 117, 110, 97, 32, 77, 97, 116, 97, 116, 97]

Decrypted message in hex format ['0x53', '0x65', '0x63', '0x6f', '0x6e', '0x64', '0x20', '0x6d', '0x65', '0x73', '0x73'